**任务时间：题目1-3（11.05-11.10）
题目4、5（11.12-11.17）
题目6（11.19-11.25）**

**互评与筛选时间：题目1-3（11.11）
题目4、5（11.18）
题目6（11.26）**

每周只用提交相应的部分到自己的git，试题实行每周淘汰制

新互评链接：https://yiqixie.com/s/home/fcACT-kZRz10FXWBU2jXDt8VO

# 6、将你的其中一个假设落实，如果能完成两个更好，但不强求

利用TimingSignal最后输出long、short、long_short这三个绩效表出来


**alpha 101 #008因子与btc价格的联系**  
alpha8因子表征了股票势头的相对强弱，当势头强到一定程度时会变弱，过弱时同理。  
从close图与008因子的图中可以看出，alpha8的尖峰（局部最大值）往往对应着close图中一个较低的价格，即释放了买入信号，而alpha8的局部最小值往往对应着close图中的一个较高的价格，释放了卖出信号。  
通过更细致的观察发现，这种相关关系在500到-500的区间内并不显著，在大于500以及小于-500时信号更加清晰。  
进场假设：alpha8出现小于-500的局部最小值  
出场假设：alpha8出现大于500的局部最大值  

In [1]:
import warnings
from jaqs_fxdayu.data.dataservice import LocalDataService
from time import time
warnings.filterwarnings('ignore')

ds = LocalDataService()

start = time()
path = r'./min_data/VnTrader_1Min_Db'
props = {'fields': 'open,high,low,close,volume',
         'symbol': 'EOSUSDT:binance',#BTCUSDT:binance，ETHUSDT:binance,EOSUSDT:binance
         'freq': '1H',
         'start_date':20180601000000}

Time_dict = ds.bar_reader(path,props) #读取数据

In [2]:
from jaqs_fxdayu.data.hf_dataview import HFDataView
dv1H = HFDataView()
dv1H.create_init_dv(Time_dict.set_index(["trade_date","symbol"]))

Initialize dataview success.


In [3]:
def RankPct(df):
    return df.rank(axis=1, pct=True)
alpha8_change = dv1H.add_formula('alpha8_C','-1*((Ts_Sum(open,5)*Ts_Sum(Return(close,1),5))-Delay((Ts_Sum(open,5)*Ts_Sum(Return(close,1),5)),10))',add_data=True, register_funcs={'RankPct':RankPct})
stddev = dv1H.add_formula("deviation", "StdDev(alpha8_C, 20)", add_data=True, register_funcs={'RankPct':RankPct})

### 对假设的修正

原来的假设：  
进场假设：alpha8出现小于-500（近似，可微调）的局部最小值  
出场假设：alpha8出现大于500（近似，可微调）的局部最大值  
  
新的假设（引入alpha8标准差）：  
出场假设：alpha8出现了小于指标标过去n（本题中设置为20）日内该准差2倍（近似，可微调）的相反数的局部最小值  
出场假设：alpha8出现了大于指标标过去n（本题中设置为20）日内该准差2倍（近似，可微调）的局部最大值  
  
由于不同币种的价格数值是不同的，如btc和eos的价格相差三个数量级，所以btc的alpha8因子值通常在200到1000左右而eos的alpha8因子值基本都小于1，以数值设定的假设显然不适用于eos，新的假设是为了适应不同币种的alpha8因子值而进行的修正  
事实上，对btc来讲，新的假设与旧的假设基于相同的策略，不过灵活性更优  

结果表明，使用了新的假设后，btc的win_ration表现和win_mean/loss_mean绩效表现略有提升（尽管原来的假设也达到了大于0.4的win_ratio和大于1的win_mean/loss_mean）

In [4]:
import pandas as pd
# 进场信号
long = dv1H.add_formula("long", "If((Return(Return(alpha8_C,1),1)>0.25) && (alpha8_C < -1*deviation*(1-0.1)*2),2,0)", add_data=True)
short = dv1H.add_formula("short", "If((Return(Return(alpha8_C,1),1)<-0.25) && (alpha8_C > deviation*(1-0.1)*2),-2,0)", add_data=True)
# 出场信号
close_long = dv1H.add_formula("closeLong","If(short==-2,1,0)", add_data=True)
close_short = dv1H.add_formula("closeShort","If(long==2,-1,0)", add_data=True)

In [5]:
#观察出现做多信号的时间点
#dv1H.get_ts('long').stack()[dv1H.get_ts('long').stack()==2]

In [6]:
#通过测试报告，看多空信号的绩效，这里是定义报告格式
from jaqs_fxdayu.research import TimingDigger

tdMa = TimingDigger(output_folder=".", output_format='pdf',signal_name='Ma')

def TimingSignal(td, dv, long='long', short='short',closeLong='closeLong', closeShort='closeShort'\
                 , mhp=None,sl=None,sp=None):
    # step 1：实例化TimingDigger 通过output_folder和output_format指定测试报告的输出路径和输出格式，通过signal_name指定绩效文件名称

    #多空信号分别计算一遍 输出汇总结果
    td.process_signal(
        enter_signal=dv.get_ts(long),
        exit_signal=dv.get_ts(closeLong),
        sig_type="long",  # 信号类型 long/short
        price=dv.get_ts('close'),
        max_holding_period=mhp,  # 最大持有天数 可为空
        stoploss=-sl,  # 止损百分比 负数 可为空
        stopprofit=sp,  # 止盈百分比 正数 可为空
    )

    td.process_signal(
        enter_signal=dv.get_ts(short),
        exit_signal=dv.get_ts(closeShort),
        sig_type="short",  # 信号类型 long/short
        price=dv.get_ts("close"),
        max_holding_period=mhp,  # 最大持有天数 可为空
        stoploss=-sl,  # 止损百分比 负数 可为空
        stopprofit=sp,  # 止盈百分比 正数 可为空
    )

In [7]:
TimingSignal(tdMa, dv1H, mhp=100, sl=0.05, sp=0.3)#调取数据

Nan Data Count (should be zero) : 0;  Percentage of effective data: 1%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 3%


In [8]:
tdMa.create_event_report(sig_type="long")

*****-Summary-*****
Event Analysis
                       win   loss     all
t-stat               4.648 -6.492   0.839
p-value              0.001  0.001   0.413
mean                 0.053 -0.052   0.012
std                  0.036  0.020   0.060
info_ratio           1.470 -2.650   0.204
skewness            -0.181  1.242   0.027
kurtosis            -1.340  0.971  -1.418
pct5                 0.002 -0.073  -0.062
pct25                0.022 -0.059  -0.054
pct50                0.059 -0.056   0.006
pct75                0.081 -0.053   0.069
pct95                0.102 -0.021   0.099
occurance           11.000  7.000  18.000
win_ratio              NaN    NaN   0.611
win_mean/loss_mean     NaN    NaN   1.019


In [9]:
tdMa.create_event_report(sig_type="short")

*****-Summary-*****
Event Analysis
                       win    loss     all
t-stat               9.531 -14.500   3.114
p-value              0.000   0.000   0.003
mean                 0.115  -0.055   0.036
std                  0.078   0.023   0.103
info_ratio           1.471  -2.417   0.350
skewness             0.395   0.318   0.649
kurtosis            -0.386  -0.302  -0.641
pct5                 0.003  -0.089  -0.076
pct25                0.039  -0.069  -0.055
pct50                0.123  -0.055   0.006
pct75                0.161  -0.051   0.125
pct95                0.219  -0.017   0.211
occurance           43.000  37.000  80.000
win_ratio              NaN     NaN   0.538
win_mean/loss_mean     NaN     NaN   2.084


In [10]:
tdMa.create_event_report(sig_type="long_short")

*****-Summary-*****
Event Analysis
                       win    loss     all
t-stat               9.839 -16.030   3.220
p-value              0.000   0.000   0.002
mean                 0.102  -0.055   0.032
std                  0.076   0.022   0.097
info_ratio           1.351  -2.445   0.327
skewness             0.617   0.409   0.717
kurtosis            -0.100  -0.122  -0.367
pct5                 0.002  -0.087  -0.076
pct25                0.035  -0.067  -0.055
pct50                0.095  -0.056   0.006
pct75                0.149  -0.051   0.107
pct95                0.215  -0.011   0.204
occurance           54.000  44.000  98.000
win_ratio              NaN     NaN   0.551
win_mean/loss_mean     NaN     NaN   1.871


### 结论

                                        long          short        long/short
混合绩效(win_ratio  win_mean/loss_mean)：0.525/1.757； 0.497/2.001； 0.502/1.954  
btc绩效(win_ratio  win_mean/loss_mean)： 0.455/2.900； 0.462/1.629； 0.461/1.810  
eos绩效(win_ratio  win_mean/loss_mean)： 0.611/1.019； 0.538/2.084； 0.551/1.871  
  
总结：假设得到验证，所有绩效均达到了win_ratio大于0.4和win_mean/loss_mean大于1的要求。  
从上表中不难看出，尽管该策略是为btc设计的，但却在eos市场获得了更好的综合绩效表现，说明策略的通用性尚可  
事实上，不同币种的价格虽然相差很大，但其本质大致相似，因此同一个策略在不同的币种中有着大致相同的表现也在意料之中